In [1]:
import pandas as pd
import numpy as np 

In [2]:

# pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTnLjDvKnVVqQfVA7QGhXXdV8Eo7fOFc_dYp4HMfqSzcTdYuSJI4GccuxeaBgzGtH7m1M9GfLZkzFST/pub?gid=2008373479&single=true&output=csv")



In [37]:
# df = pd.read_csv('interior_design_qa.csv')
# df = pd.read_csv('intents_qa.csv')
df=pd.read_csv('qa_pairs.csv')

In [38]:
df

,Question,Answer
0,Hi there,Hello! Am Bot Here to help you.
1,How are you,Good to see you again! Am Bot Here to help you.
2,Is anyone there?,"Hi there, how can I help?"
3,Hey,Hello! Am Bot Here to help you.
4,Hola,Hello! Am Bot Here to help you.
...,...,...
74,Are you expensive?,I'm a free virtual assistant provided for info...
75,Who’s your boss / master?,I don't have a boss or master. I operate based...
76,Do you get smarter?,I don't inherently get smarter on my own. My p...
77,Address,Delhi Laxmi Interior is located near [Include ...


In [39]:
questions_list = df['Question'].tolist()
answers_list = df['Answer'].tolist()

In [18]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [19]:
!pip install pyaspeller

Defaulting to user installation because normal site-packages is not writeable


In [40]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re 

In [41]:
def preprocess(text):
    lemmatizer = WordNetLemmatizer() 
    stemmer =  PorterStemmer() 
    text = re.sub(r'[^\w\s]','',text)  # removing non alphanumeric characters 
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens  = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens  = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [42]:
def preprocess_with_stopwords(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [10]:
# vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
# X = vectorizer.fit_transform([preprocess(q) for q in questions_list])

In [11]:
# # -----------------Not using -------------------------

# # Function to calculate chatbot response
# def chatbot_response(input_text):
#     processed_input = preprocess_with_stopwords(input_text)

#     # Use the same vectorizer instance for both X and vectorized_input
#     vectorized_input = vectorizer.transform([processed_input])

#     similarities = cosine_similarity(vectorized_input, X)
#     max_similarity = np.max(similarities)

#     if max_similarity > 0.6:
#         high_similarity_questions = [q for q, s in zip(questions_list, similarities[0]) if s > 0.6]
#         target_answers = [answers_list[questions_list.index(q)] for q in high_similarity_questions]

#         # Use the same vectorizer instance for both Z and vectorized_input_with_stopwords
#         Z = vectorizer.transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
#         vectorized_input_with_stopwords = vectorizer.transform([processed_input])

#         final_similarities = cosine_similarity(vectorized_input_with_stopwords, Z)
#         closest = np.argmax(final_similarities)

#         return target_answers[closest]
#     else:
#         return "I'm sorry, I don't understand. Could you please rephrase or provide more details?"

# # Sample usage
# user_input = "lvng room design ideas"
# response = chatbot_response(user_input)
# print(response) 

In [46]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
X = vectorizer.fit_transform([preprocess(q) for q in questions_list])
def chatbot_response(input_text):
    processed_input = preprocess(input_text)
    vectorized_input = vectorizer.transform([processed_input])

    similarities = cosine_similarity(vectorized_input, X)
    max_similarity = np.max(similarities)

    # Dynamic threshold based on the 75th percentile
    threshold = np.percentile(similarities, 75)

    if max_similarity > threshold:
        high_similarity_questions = [q for q, s in zip(questions_list, similarities[0]) if s > threshold]
        target_answers = [answers_list[questions_list.index(q)] for q in high_similarity_questions]

        Z = vectorizer.transform([preprocess(q) for q in high_similarity_questions])
        final_similarities = cosine_similarity(vectorized_input, Z)
        closest = np.argmax(final_similarities)

        return target_answers[closest]
    else:
        return "To know more call/whatsapp us on +918779693725 or email us at ankitmishra.letter@gmail.com"

# Sample usage
user_input = "what are your services"
response = chatbot_response(user_input)
print(response)

We offer a range of services, including residential and commercial interior design, space planning, color consultation, and furniture selection. Our goal is to create spaces that reflect your style and meet your functional needs.


In [48]:
while True:
    user_input = input("You: ")
    
    # Exit the loop if the user types "exit" or "quit"
    if user_input.lower() in ['exit', 'quit']:
        print("Exiting the chat.")
        break
    
    print("You:", user_input)
    
    response = chatbot_response(user_input)
    print("Chatbot:", response)

You: hey
Chatbot: Hello! Am Bot Here to help you.
You: hey
Chatbot: Hello! Am Bot Here to help you.
You: what are your services
Chatbot: We offer a range of services, including residential and commercial interior design, space planning, color consultation, and furniture selection. Our goal is to create spaces that reflect your style and meet your functional needs.
You: 
Chatbot: To know more call/whatsapp us on +918779693725 or email us at ankitmishra.letter@gmail.com
You: address ?
Chatbot: Delhi Laxmi Interior is located near [Include Specific Location Details]. For more precise directions, you may want to use a map or contact them directly.
You: 
Chatbot: To know more call/whatsapp us on +918779693725 or email us at ankitmishra.letter@gmail.com
You: give contact info
Chatbot: You can reach us at [Your Contact Number]. Feel free to call us or send an email to [Your Email Address] for inquiries or to schedule a consultation.
You: 
Chatbot: To know more call/whatsapp us on +91877969372

In [13]:
import string

In [14]:
from pyaspeller import YandexSpeller

def error_correct_pyspeller(sample_text):
    speller = YandexSpeller()
    fixed = speller.spelled(sample_text)
    return fixed

input_text = """
 what is data anlytics ?
"""


output_text = error_correct_pyspeller(input_text)
print(output_text)


  what is data analytics ?



In [15]:
import Caribe as cb


sentence = "wha is hi nae"
standard = cb.caribe_corrector(sentence)

print(standard) # Outputs: I am playing football outside

What's up?


In [16]:
from pyaspeller import YandexSpeller
import language_tool_python

def error_correcting(text):
    tool = language_tool_python.LanguageTool('en-US')
    datasets = tool.correct(text)
    return datasets

def error_correct_pyspeller(sample_text):
    speller = YandexSpeller()
    fixed = speller.spelled(sample_text)
    return fixed

input_text = """
This is a sample paragrap with some incorrect spellings and grammer mistaks.
It's importnt to check larje text chunks for accurcy and improve readibility.
Gingerit is a great library for such tasks, and it can handl larje text as well.

Let's try processing this larje text using Gingerit.
"""

# output_data = error_correcting(input_text)
# print(output_data)

output_text = error_correct_pyspeller(input_text)
print(output_text)


This is a sample paragraph with some incorrect spelling and grammar mistakes.
It's important to check large text chunks for accuracy and improve readability.
Ginger it is a great library for such tasks, and it can handle large text as well.

Let's try processing this large text using Ginger it.



In [17]:
# from gingerit.gingerit import GingerIt

# text = 'What is Data Anlytics'

# parser = GingerIt()
# corrected_text = parser.parse(text)

# print(corrected_text['result'])

1st Concept 

#####  Some question who has no match will be saved in other file, 
##### Training Data is not available 


2nd Concept 


#####  List of Qs & Answers 
using LLM & keep storing those qs
